In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--remote-debugging-port=9222")

# ChromeDriver 설정 및 브라우저 실행
service = Service(
    ChromeDriverManager().install(),
    log_path="chromedriver.log"  # 로그 파일 저장 경로
)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# 웹 페이지 열기
driver.get("https://heroesbaseball.co.kr/games/schedule/list1st.do")

# 페이지 로딩 대기
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'searchYear')))

# 연도와 월 설정 및 이동 버튼 클릭
driver.execute_script("""
    $('#searchYear').val('2024').trigger('change');
    $('#searchMonth').val('9').trigger('change');
    $('.submit').click();
""")

# 경기 데이터가 로드될 때까지 대기
try:
    WebDriverWait(driver, 40).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".gameList td a"))
    )
except Exception as e:
    print("경기 데이터 로드 실패:", e)
    driver.quit()
    exit()

# 경기 정보 추출
game_cells = driver.find_elements(By.CSS_SELECTOR, ".gameList td a")

# DOM 확인용 출력
print("DOM 상태 확인:")
print(driver.execute_script("return document.body.innerHTML;"))

if not game_cells:
    print("경기 데이터가 없습니다.")
else:
    for game in game_cells:
        try:
            # 팀 이름 및 점수 추출
            teams = game.find_elements(By.TAG_NAME, "strong")
            home_team = teams[0].text.split("<em>")[0].strip()
            away_team = teams[1].text.split("<em>")[0].strip()

            home_score = teams[0].find_element(By.TAG_NAME, "em").text
            away_score = teams[1].find_element(By.TAG_NAME, "em").text

            # 경기 장소 및 시간
            location_time = game.find_element(By.CLASS_NAME, "location").text.strip()

            # 결과 추출
            result_img = game.find_element(By.CLASS_NAME, "result")
            result = result_img.get_attribute("alt")  # '승', '패', '무'

            # 출력
            print(f"경기: {home_team} {home_score} : {away_team} {away_score}")
            print(f"결과: {result}")
            print(f"장소 및 시간: {location_time}")
            print("-" * 30)
        except Exception as e:
            print(f"오류 발생: {e}")

# 브라우저 종료
driver.quit()


경기 데이터 로드 실패: Message: 



MaxRetryError: HTTPConnectionPool(host='localhost', port=50883): Max retries exceeded with url: /session/2b4e44e859e085edc08e62c5b9d68af3/elements (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002CA257F2E50>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

: 

In [1]:
from selenium.webdriver.chrome.service import Service
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

NameError: name 'webdriver' is not defined

scheduleList
    날짜
        vscore
        gyear
        gmonth
        hscore
        gtime
        stadium
        cancel_flag
        gday
        home
        gamedate
        gweek
        visit


In [ ]:
import requests
import csv

# 요청 헤더 설정
headers = {
    "referer": "https://heroesbaseball.co.kr/games/schedule/list1st.do",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"
}

# API 요청
url = "https://heroesbaseball.co.kr/games/schedule/getSports2iScheduleList.do?searchYear=2024&searchMonth=08&flag=1"
response = requests.get(url, headers=headers)

# JSON 데이터를 딕셔너리로 변환
if response.status_code == 200:
    data = response.json()  # JSON 데이터를 딕셔너리로 변환
    games_list = data.get("scheduleList", [])  # 경기 정보 리스트 추출

    if games_list:
        # CSV 파일 저장 경로
        file_path = "games/8월_경기결과.csv"

        # CSV 파일 저장
        with open(file_path, mode="w", encoding="utf-8-sig", newline="") as file:
            writer = csv.writer(file)
            # 헤더 작성
            writer.writerow(["월", "날짜", "시간", "홈 팀", "홈 점수", "원정 팀", "원정 점수", "경기장", "상태"])
            
            for game in games_list:
                # 필요한 정보 추출 
                gmonth = game.get("gmonth")
                gday = game.get("gday")  # 날짜
                gtime = game.get("gtime")  # 시간
                home_team = game.get("home")  # 홈 팀
                away_team = game.get("visit")  # 원정 팀
                home_score = game.get("hscore", 0)  # 홈 팀 점수 (기본값 0)
                away_score = game.get("vscore", 0)  # 원정 팀 점수 (기본값 0)
                stadium = game.get("stadium", "미정")  # 경기장 (기본값 "미정")
                result = "진행 중" if game.get("end_flag") == "0" else "종료"

                # 데이터 행 작성
                writer.writerow([f"2024-08-{gday}", gtime, home_team, home_score, away_team, away_score, stadium, result])
                
        print(f"{file_path} 파일로 저장 완료!")
    else:
        print("경기 데이터가 없습니다.")
else:
    print(f"API 요청 실패, 상태 코드: {response.status_code}")


날짜: 2024-08-01
시간: 18:30
경기: 키움 14 vs NC 9
경기장: 고척
상태: 종료
------------------------------
날짜: 2024-08-02
시간: 18:30
경기: 두산 4 vs 키움 6
경기장: 잠실
상태: 종료
------------------------------
날짜: 2024-08-03
시간: 18:00
경기: 두산 5 vs 키움 15
경기장: 잠실
상태: 종료
------------------------------
날짜: 2024-08-04
시간: 17:00
경기: 두산 0 vs 키움 0
경기장: 잠실
상태: 진행 중
------------------------------
날짜: 2024-08-06
시간: 18:30
경기: 키움 2 vs SSG 6
경기장: 고척
상태: 종료
------------------------------
날짜: 2024-08-07
시간: 18:30
경기: 키움 2 vs SSG 6
경기장: 고척
상태: 종료
------------------------------
날짜: 2024-08-08
시간: 18:30
경기: 키움 3 vs SSG 0
경기장: 고척
상태: 종료
------------------------------
날짜: 2024-08-09
시간: 18:30
경기: 한화 7 vs 키움 5
경기장: 대전
상태: 종료
------------------------------
날짜: 2024-08-10
시간: 18:00
경기: 한화 1 vs 키움 3
경기장: 대전
상태: 종료
------------------------------
날짜: 2024-08-11
시간: 18:00
경기: 한화 3 vs 키움 7
경기장: 대전
상태: 종료
------------------------------
날짜: 2024-08-13
시간: 18:30
경기: 키움 0 vs KIA 2
경기장: 고척
상태: 종료
------------------------------
날짜: 2024-08-14
시간: 18:30